# Excercise 4: Containerize Model

Now that we have a model, we can build a container around it.

## Excercise 1: Evaluate Dummy Service 

Currently our frontend calls a dummy service to check if a user is eligible. 

Our dummy service already provides the basic infrastrucure to serve a real sklearn model.

Read through the `DummyModel.py`, `requirements.txt` and the `Dockerfile` at `services/dummy` and try to understand whats going on.

Questions
- What is the predicion of our dummy service?
- How could you adapt our dummy service to serve our model?

## Excercise 2: Create wrapper around our model

Now that we understand our dummy service, we want to load our previously trained model. 

1. Copy dummy service
  
  `services/dummy/DummyModel.py`,`services/dummy/requirements.txt` and `services/dummy/Dockerfile` to 
  `services/eligibilty/EligibilityModel.py`, `services/eligibilty/requirements.txt` and `services/eligibilty/Dockerfile`.
2. Copy our model from `notebooks/model/elibility_model.joblib` to `services/eligibilty/elibility_model.joblib`
3. Adjust `EligibilityModel.py` to load our pipeline from the joblib file and call our pipeline. Use the function `load` from joblib to load our model. See https://scikit-learn.org/stable/modules/model_persistence.html#python-specific-serialization




## Excercise 3: Build docker image and start it

To serve our model from Docker we need to adjust the Dockerfile at `services/eligibilty/Dockerfile`:

Change `ENV MODEL_NAME DummyModel` to `ENV MODEL_NAME EligibilityModel`

And adjust the `docker-compose.yml` to use our new elibility service instead of dummy:

Change 
```yml
eligibility-service:
    build: ./services/dummy
``` 
to 

```yml
eligibility-service:
    build: ./services/eligibility
``` 

Then rebuild the eligibility-serviec with docker-compose: `docker-compose down`, `docker-compose build eligibility-service` and `docker-compose up`

## Excercise 4: Call our eligibility service

Our eligibility service now exposes our model using a REST API. We can call it:


In [6]:
!wget -O- http://eligibility-service:9000/predict --post-data '{"data": { "ndarray": [[25000, 10000]]}}' --header='Content-Type:application/json'

--2021-08-09 13:45:04--  http://eligibility-service:9000/predict
Resolving eligibility-service (eligibility-service)... 127.0.0.1, 127.0.0.1, 127.0.0.1, ...
Connecting to eligibility-service (eligibility-service)|127.0.0.1|:9000... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46 [application/json]
Saving to: ‘STDOUT’


-                     0%[                    ]       0  --.-KB/s               {"data":{"names":[],"ndarray":[0]},"meta":{}}

-                   100%[===================>]      46  --.-KB/s    in 0s      

2021-08-09 13:45:04 (9.27 MB/s) - written to stdout [46/46]



If you receive a succssful response "200 OK" you can switch to our frontend and test our service like an end user: http://localhost:8000/#order